# Feature extraction for all wavelets
- This notebook extracts features from all wavelets, where the DWT is performed after event isolation
- uses data from event_generation_copy.ipynb

In [2]:
import importlib
import all_funcs
importlib.reload(all_funcs)
from all_funcs import *

Function to apply all types of wavelets to data

In [129]:
def DWT_and_features_thresh_trace(event_time, event_data, mean_noise, sd_threshold, sd_threshold_lower, NP_label, wavelet, threshold, NRNS = False):
    """DWT_and_features with thresholding and peak tracing

    Args:
        event_time, event_data (list of arrays): event data for 1 run, containing all events 
        mean_noise, sd_threshold, sd_threshold_lower, NP_size, wavelet, threshold

    Returns:
        DWT_rec, event_time_padded, DWT_rec_padded, features_df, features_list, labels, all_coeffs
    """
    
    
    DWT_rec, all_coeffs = zip(*[lowpassfilter_all(signal, wavelet=wavelet, thresh = threshold) for signal in event_data])
    DWT_rec, all_coeffs = list(DWT_rec), list(all_coeffs)
    
    DWT_rec = peak_tracer(event_data, DWT_rec, sd_threshold, sd_threshold_lower, thresh = threshold, NRNS = NRNS)
    
    event_time_padded, DWT_rec_padded = pad_event(event_time, DWT_rec, mean_noise)
    features_df, features_list = find_features_CWT(event_time_padded, DWT_rec_padded, mean_noise, all_coeffs, sd_threshold, sd_threshold_lower)
    
    # label 0 for 5nm or NS, 1 for 10nm or NR
    labels = np.ones(len(features_list)) if NP_label in [10, "NR"] else np.zeros(len(features_list))

    
    
    return DWT_rec, event_time_padded, DWT_rec_padded, features_df, features_list, labels, all_coeffs


In [2]:
event_times_all_wavelets = unpickle("data_1_to_20_df3.pkl")["event_time"]
event_datas_all_wavelets = unpickle("data_1_to_20_df3.pkl")["event_data"]
mean_noises_all_wavelets = unpickle("data_1_to_20_df3.pkl")["mean_noise"]
sd_thresholds_all_wavelets = unpickle("data_1_to_20_df3.pkl")["sd_threshold"]
sd_thresholds_lower_all_wavelets = unpickle("data_1_to_20_df3.pkl")["sd_threshold_lower"]

NP_sizes_all_wavelets = [5,5,5,5,5,5,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]


### Feature extraction on all wavelets

In [2]:
def all_wavelets(event_times, event_datas, mean_noises, sd_thresholds, sd_thresholds_lower, NP_sizes, threshold = 0.1):
    wavelets = pywt.wavelist(kind='discrete')  # Use first 3 wavelets for testing
    df_list = []
    event_time_padded_dic = {}
    DWT_rec_padded_dic = {}
    df_events_all_wavelets = pd.DataFrame(columns=wavelets)

    # Generate the ordered list of keys for rows
    ordered_keys = []
    idx = 2
    prev_size = 5
    for NP_size in NP_sizes:
        if NP_size - prev_size == 5:
            idx = 2
        ordered_keys.append(f'DWT_{NP_size}nm_{idx}')
        prev_size = NP_size
        idx += 1

    for wavelet in wavelets:
        event_time_padded_list_wavelet = []
        DWT_rec_padded_list_wavelet = []
        
        idx = 2
        prev_size = 5
        row = {key: None for key in ordered_keys}  # Initialize row with ordered keys
        for event_index, (key, (event_time, event_data, mean_noise, sd_threshold, sd_threshold_lower, NP_size)) in enumerate(zip(ordered_keys, zip(event_times, event_datas, mean_noises, sd_thresholds, sd_thresholds_lower, NP_sizes))):
            try:
                if NP_size - prev_size == 5:
                    idx = 2

                DWT_rec, event_time_padded, DWT_rec_padded, features_df, features_list, labels, all_coeffs = DWT_and_features_thresh_trace(event_time, event_data, mean_noise, sd_threshold, sd_threshold_lower, NP_size, wavelet, threshold, NRNS = False)
                
                event_time_padded_list_wavelet.append(event_time_padded)
                DWT_rec_padded_list_wavelet.append(DWT_rec_padded)

                row[key] = features_list
                prev_size = NP_size
                idx += 1

            except Exception as e:
                print(f"Error occurred for event index {event_index}, wavelet '{wavelet}': {e}")
                # Optionally, handle the error further or re-raise it
                # raise
                
        event_time_padded_dic[wavelet] = event_time_padded_list_wavelet
        DWT_rec_padded_dic[wavelet] = DWT_rec_padded_list_wavelet
        
        df_events_all_wavelets.loc['event_time_padded', wavelet] = event_time_padded_dic[wavelet]
        df_events_all_wavelets.loc['DWT_rec_padded', wavelet] = DWT_rec_padded_dic[wavelet]
        
        # Create DataFrame with explicit row order
        wavelet_df = pd.DataFrame({wavelet: row}, index=ordered_keys)
        df_list.append(wavelet_df)
    

        
    # features
    # Concatenate all DataFrames in the list to form the final DataFrame
    df_all_wavelets = pd.concat(df_list, axis=1)
    
    return df_all_wavelets, df_events_all_wavelets


##### Without coeffs

In [57]:
results_all_wavelets_thresh3005, df_events_all_wavelets_thresh3005 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.05)

In [58]:
results_all_wavelets_thresh3005.to_pickle("results_all_wavelets_thresh3005.pkl")

In [59]:
results_all_wavelets_thresh301, df_events_all_wavelets_thresh301 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.1)

In [60]:
results_all_wavelets_thresh301.to_pickle("results_all_wavelets_thresh301.pkl")

In [61]:
results_all_wavelets_thresh3015, df_events_all_wavelets_thresh3015 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.15)

In [62]:
results_all_wavelets_thresh3015.to_pickle("results_all_wavelets_thresh3015.pkl")

In [63]:
results_all_wavelets_thresh302, df_events_all_wavelets_thresh302 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.2)

In [64]:
results_all_wavelets_thresh302.to_pickle("results_all_wavelets_thresh302.pkl")

In [65]:
results_all_wavelets_thresh3025, df_events_all_wavelets_thresh3025 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.25)

In [66]:
results_all_wavelets_thresh3025.to_pickle("results_all_wavelets_thresh3025.pkl")

In [67]:
results_all_wavelets_thresh303, df_events_all_wavelets_thresh303 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.3)

In [68]:
results_all_wavelets_thresh303.to_pickle("results_all_wavelets_thresh303.pkl")

In [69]:
results_all_wavelets_thresh3035, df_events_all_wavelets_thresh3035 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.35)

In [70]:
results_all_wavelets_thresh3035.to_pickle("results_all_wavelets_thresh3035.pkl")

In [71]:
results_all_wavelets_thresh304, df_events_all_wavelets_thresh304 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.4)

In [72]:
results_all_wavelets_thresh304.to_pickle("results_all_wavelets_thresh304.pkl")

In [73]:
results_all_wavelets_thresh3045, df_events_all_wavelets_thresh3045 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.45)

In [74]:
results_all_wavelets_thresh3045.to_pickle("results_all_wavelets_thresh3045.pkl")

In [75]:
results_all_wavelets_thresh305, df_events_all_wavelets_thresh305 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.5)

In [76]:
results_all_wavelets_thresh305.to_pickle("results_all_wavelets_thresh305.pkl")

##### With coeffs

In [ ]:
# thresh = 0.05
results_all_wavelets_thresh005_coeff2, df_events_all_wavelets_thresh005_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.05)


In [169]:
results_all_wavelets_thresh005_coeff2.to_pickle("results_all_wavelets_thresh005_coeff2.pkl")

In [ ]:
df_events_all_wavelets_thresh005_coeff2.to_pickle("df_events_all_wavelets_thresh005_coeff2.pkl")

In [36]:
results_all_wavelets_thresh01_coeff2, df_events_all_wavelets_thresh01_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.1)


In [21]:
results_all_wavelets_thresh01_coeff2.to_pickle("results_all_wavelets_thresh01_coeff2.pkl")

In [37]:
df_events_all_wavelets_thresh01_coeff2.to_pickle("df_events_all_wavelets_thresh01_coeff2.pkl")

In [38]:
# thresh = 0.15
results_all_wavelets_thresh015_coeff2, df_events_all_wavelets_thresh015_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.15)


In [167]:
results_all_wavelets_thresh015_coeff2.to_pickle("results_all_wavelets_thresh015_coeff2.pkl")

In [39]:
df_events_all_wavelets_thresh015_coeff2.to_pickle("df_events_all_wavelets_thresh015_coeff2.pkl")

In [6]:
# thresh = 0.2
results_all_wavelets_thresh02_coeff2, df_events_all_wavelets_thresh02_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.2)


In [165]:
results_all_wavelets_thresh02_coeff2.to_pickle("results_all_wavelets_thresh02_coeff2.pkl")

In [7]:
df_events_all_wavelets_thresh02_coeff2.to_pickle("df_events_all_wavelets_thresh02_coeff2.pkl")

In [8]:
# thresh = 0.25
results_all_wavelets_thresh025_coeff2, df_events_all_wavelets_thresh025_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.25)


In [171]:
results_all_wavelets_thresh025_coeff2.to_pickle("results_all_wavelets_thresh025_coeff2.pkl")

In [ ]:
df_events_all_wavelets_thresh025_coeff2.to_pickle("df_events_all_wavelets_thresh025_coeff2.pkl")

In [41]:
# thresh = 0.3
results_all_wavelets_thresh03_coeff2, df_events_all_wavelets_thresh03_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.3)


In [173]:
results_all_wavelets_thresh03_coeff2.to_pickle("results_all_wavelets_thresh03_coeff2.pkl")

In [42]:
df_events_all_wavelets_thresh03_coeff2.to_pickle("df_events_all_wavelets_thresh03_coeff2.pkl")

In [ ]:
# thresh = 0.35
results_all_wavelets_thresh035_coeff2, df_events_all_wavelets_thresh035_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.35)


In [175]:
results_all_wavelets_thresh035_coeff2.to_pickle("results_all_wavelets_thresh035_coeff2.pkl")

In [ ]:
df_events_all_wavelets_thresh035_coeff2.to_pickle("df_events_all_wavelets_thresh035_coeff2.pkl")

In [ ]:
# thresh = 0.4
results_all_wavelets_thresh04_coeff2, df_events_all_wavelets_thresh04_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.4)


In [177]:
results_all_wavelets_thresh04_coeff2.to_pickle("results_all_wavelets_thresh04_coeff2.pkl")

In [ ]:
df_events_all_wavelets_thresh04_coeff2.to_pickle("df_events_all_wavelets_thresh04_coeff2.pkl")

In [22]:
# thresh = 0.45
results_all_wavelets_thresh045_coeff2, df_events_all_wavelets_thresh045_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.45)


In [23]:
results_all_wavelets_thresh045_coeff2.to_pickle("results_all_wavelets_thresh045_coeff2.pkl")

In [24]:
# thresh = 0.5
results_all_wavelets_thresh05_coeff2, df_events_all_wavelets_thresh05_coeff2 = all_wavelets(event_times_all_wavelets, event_datas_all_wavelets, mean_noises_all_wavelets, sd_thresholds_all_wavelets, sd_thresholds_lower_all_wavelets, NP_sizes_all_wavelets, threshold = 0.5)


In [25]:
results_all_wavelets_thresh05_coeff2.to_pickle("results_all_wavelets_thresh05_coeff2.pkl")

In [35]:
df_events_all_wavelets_thresh045_coeff2.to_pickle("df_events_all_wavelets_thresh045_coeff2.pkl")
df_events_all_wavelets_thresh05_coeff2.to_pickle("df_events_all_wavelets_thresh05_coeff2.pkl")

saving

In [112]:
# features
results_all_wavelets_thresh01.to_pickle("results_all_wavelets_thresh01.pkl")
results_all_wavelets_thresh02.to_pickle("results_all_wavelets_thresh02.pkl")
results_all_wavelets_thresh03.to_pickle("results_all_wavelets_thresh03.pkl")
results_all_wavelets_thresh04.to_pickle("results_all_wavelets_thresh04.pkl")

In [113]:
results_all_wavelets_thresh005.to_pickle("results_all_wavelets_thresh005.pkl")
results_all_wavelets_thresh015.to_pickle("results_all_wavelets_thresh015.pkl")
results_all_wavelets_thresh025.to_pickle("results_all_wavelets_thresh025.pkl")
results_all_wavelets_thresh035.to_pickle("results_all_wavelets_thresh035.pkl")
results_all_wavelets_thresh045.to_pickle("results_all_wavelets_thresh045.pkl")
results_all_wavelets_thresh05.to_pickle("results_all_wavelets_thresh05.pkl")

In [ ]:
results_all_wavelets_thresh2005.to_pickle("results_all_wavelets_thresh2005.pkl")
results_all_wavelets_thresh201.to_pickle("results_all_wavelets_thresh201.pkl")
results_all_wavelets_thresh2015.to_pickle("results_all_wavelets_thresh2015.pkl")
results_all_wavelets_thresh202.to_pickle("results_all_wavelets_thresh202.pkl")
results_all_wavelets_thresh2025.to_pickle("results_all_wavelets_thresh2025.pkl")
results_all_wavelets_thresh203.to_pickle("results_all_wavelets_thresh203.pkl")
results_all_wavelets_thresh2035.to_pickle("results_all_wavelets_thresh2035.pkl")
results_all_wavelets_thresh204.to_pickle("results_all_wavelets_thresh204.pkl")
results_all_wavelets_thresh2045.to_pickle("results_all_wavelets_thresh2045.pkl")
results_all_wavelets_thresh205.to_pickle("results_all_wavelets_thresh205.pkl")


In [ ]:
results_all_wavelets_thresh3005.to_pickle("results_all_wavelets_thresh2005.pkl")
results_all_wavelets_thresh301.to_pickle("results_all_wavelets_thresh201.pkl")
results_all_wavelets_thresh3015.to_pickle("results_all_wavelets_thresh2015.pkl")
results_all_wavelets_thresh302.to_pickle("results_all_wavelets_thresh202.pkl")
results_all_wavelets_thresh3025.to_pickle("results_all_wavelets_thresh2025.pkl")
results_all_wavelets_thresh303.to_pickle("results_all_wavelets_thresh203.pkl")
results_all_wavelets_thresh3035.to_pickle("results_all_wavelets_thresh2035.pkl")
results_all_wavelets_thresh304.to_pickle("results_all_wavelets_thresh204.pkl")
results_all_wavelets_thresh3045.to_pickle("results_all_wavelets_thresh2045.pkl")
results_all_wavelets_thresh305.to_pickle("results_all_wavelets_thresh205.pkl")


In [114]:
results_all_wavelets_thresh01 = unpickle("results_all_wavelets_thresh01.pkl")
results_all_wavelets_thresh02 = unpickle("results_all_wavelets_thresh02.pkl")
results_all_wavelets_thresh03 = unpickle("results_all_wavelets_thresh03.pkl")
results_all_wavelets_thresh04 = unpickle("results_all_wavelets_thresh04.pkl")


In [115]:
results_all_wavelets_thresh005 = unpickle("results_all_wavelets_thresh005.pkl")
results_all_wavelets_thresh015 = unpickle("results_all_wavelets_thresh015.pkl")
results_all_wavelets_thresh025 = unpickle("results_all_wavelets_thresh025.pkl")
results_all_wavelets_thresh035 = unpickle("results_all_wavelets_thresh035.pkl")
results_all_wavelets_thresh045 = unpickle("results_all_wavelets_thresh045.pkl")
results_all_wavelets_thresh05 = unpickle("results_all_wavelets_thresh05.pkl")

In [35]:
results_all_wavelets_thresh2005 = unpickle("results_all_wavelets_thresh2005.pkl")
results_all_wavelets_thresh201 = unpickle("results_all_wavelets_thresh201.pkl")
results_all_wavelets_thresh2015 = unpickle("results_all_wavelets_thresh2015.pkl")
results_all_wavelets_thresh202 = unpickle("results_all_wavelets_thresh202.pkl")
results_all_wavelets_thresh2025 = unpickle("results_all_wavelets_thresh2025.pkl")
results_all_wavelets_thresh203 = unpickle("results_all_wavelets_thresh203.pkl")
results_all_wavelets_thresh2035 = unpickle("results_all_wavelets_thresh2035.pkl")
results_all_wavelets_thresh204 = unpickle("results_all_wavelets_thresh204.pkl")
results_all_wavelets_thresh2045 = unpickle("results_all_wavelets_thresh2045.pkl")
results_all_wavelets_thresh205 = unpickle("results_all_wavelets_thresh205.pkl")


In [36]:
results_all_wavelets_thresh3005 = unpickle("results_all_wavelets_thresh3005.pkl")
results_all_wavelets_thresh301 = unpickle("results_all_wavelets_thresh301.pkl")
results_all_wavelets_thresh3015 = unpickle("results_all_wavelets_thresh3015.pkl")
results_all_wavelets_thresh302 = unpickle("results_all_wavelets_thresh302.pkl")
results_all_wavelets_thresh3025 = unpickle("results_all_wavelets_thresh3025.pkl")
results_all_wavelets_thresh303 = unpickle("results_all_wavelets_thresh303.pkl")
results_all_wavelets_thresh3035 = unpickle("results_all_wavelets_thresh3035.pkl")
results_all_wavelets_thresh304 = unpickle("results_all_wavelets_thresh304.pkl")
results_all_wavelets_thresh3045 = unpickle("results_all_wavelets_thresh3045.pkl")
results_all_wavelets_thresh305 = unpickle("results_all_wavelets_thresh305.pkl")


In [26]:
results_all_wavelets_thresh005_coeff2 = unpickle("results_all_wavelets_thresh005_coeff2.pkl")
results_all_wavelets_thresh01_coeff2 = unpickle("results_all_wavelets_thresh01_coeff2.pkl")
results_all_wavelets_thresh015_coeff2 = unpickle("results_all_wavelets_thresh015_coeff2.pkl")
results_all_wavelets_thresh02_coeff2 = unpickle("results_all_wavelets_thresh02_coeff2.pkl")
results_all_wavelets_thresh025_coeff2 = unpickle("results_all_wavelets_thresh025_coeff2.pkl")
results_all_wavelets_thresh03_coeff2 = unpickle("results_all_wavelets_thresh03_coeff2.pkl")
results_all_wavelets_thresh035_coeff2 = unpickle("results_all_wavelets_thresh035_coeff2.pkl")
results_all_wavelets_thresh04_coeff2 = unpickle("results_all_wavelets_thresh04_coeff2.pkl")
results_all_wavelets_thresh045_coeff2 = unpickle("results_all_wavelets_thresh045_coeff2.pkl")
results_all_wavelets_thresh05_coeff2 = unpickle("results_all_wavelets_thresh05_coeff2.pkl")



In [49]:
results_all_wavelets2.to_pickle("all_wavelets_df2.pkl")

In [75]:
results_all_wavelets22 = pd.read_pickle("all_wavelets_df2.pkl")

In [28]:
#old
results_all_wavelets.to_pickle("all_wavelets_df.pkl")

In [98]:
new_column_df.to_pickle("noDWT_column_df.pkl")

new

In [6]:
def results_df_combine_flat(results_df, noDWT_all_features_list, drop = False):
    
    new_column_df = pd.DataFrame({'no_DWT': noDWT_all_features_list}, index=results_df.index)

    df_all_wavelets_and_noDWT_features = results_df.merge(new_column_df,  left_index=True, right_index=True)
    
    if drop:
        df_all_wavelets_and_noDWT_features = df_all_wavelets_and_noDWT_features.drop(['DWT_10nm_19', 'DWT_10nm_20'])

    new_data_cols = {}
    for column in df_all_wavelets_and_noDWT_features.columns:
        combined_list = []
        for sublist in df_all_wavelets_and_noDWT_features[column]:
            combined_list.extend(sublist)
        new_data_cols[column] = [combined_list]

    new_df_flat = pd.DataFrame(new_data_cols)
    
    return new_df_flat

In [116]:
results_all_wavelets_thresh01_flat = results_df_combine_flat(results_all_wavelets_thresh01, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh02_flat = results_df_combine_flat(results_all_wavelets_thresh02, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh03_flat = results_df_combine_flat(results_all_wavelets_thresh03, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh04_flat = results_df_combine_flat(results_all_wavelets_thresh04, noDWT_all_features_list, drop = True)

In [117]:
results_all_wavelets_thresh005_flat = results_df_combine_flat(results_all_wavelets_thresh005, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh015_flat = results_df_combine_flat(results_all_wavelets_thresh015, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh025_flat = results_df_combine_flat(results_all_wavelets_thresh025, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh035_flat = results_df_combine_flat(results_all_wavelets_thresh035, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh045_flat = results_df_combine_flat(results_all_wavelets_thresh045, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh05_flat = results_df_combine_flat(results_all_wavelets_thresh05, noDWT_all_features_list, drop = True)


In [50]:
results_all_wavelets_thresh2005_flat = results_df_combine_flat(results_all_wavelets_thresh2005, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh201_flat = results_df_combine_flat(results_all_wavelets_thresh201, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh2015_flat = results_df_combine_flat(results_all_wavelets_thresh2015, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh202_flat = results_df_combine_flat(results_all_wavelets_thresh202, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh2025_flat = results_df_combine_flat(results_all_wavelets_thresh2025, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh203_flat = results_df_combine_flat(results_all_wavelets_thresh203, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh2035_flat = results_df_combine_flat(results_all_wavelets_thresh2035, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh204_flat = results_df_combine_flat(results_all_wavelets_thresh204, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh2045_flat = results_df_combine_flat(results_all_wavelets_thresh2045, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh205_flat = results_df_combine_flat(results_all_wavelets_thresh205, noDWT_all_features_list, drop = True)


In [51]:
results_all_wavelets_thresh3005_flat = results_df_combine_flat(results_all_wavelets_thresh2005, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh301_flat = results_df_combine_flat(results_all_wavelets_thresh201, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh3015_flat = results_df_combine_flat(results_all_wavelets_thresh2015, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh302_flat = results_df_combine_flat(results_all_wavelets_thresh202, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh3025_flat = results_df_combine_flat(results_all_wavelets_thresh2025, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh303_flat = results_df_combine_flat(results_all_wavelets_thresh203, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh3035_flat = results_df_combine_flat(results_all_wavelets_thresh2035, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh304_flat = results_df_combine_flat(results_all_wavelets_thresh204, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh3045_flat = results_df_combine_flat(results_all_wavelets_thresh2045, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh305_flat = results_df_combine_flat(results_all_wavelets_thresh205, noDWT_all_features_list, drop = True)


In [105]:
# coeff
results_all_wavelets_thresh005_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh005_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh01_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh01_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh015_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh015_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh02_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh02_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh025_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh025_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh03_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh03_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh035_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh035_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh04_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh04_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh045_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh045_coeff, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh05_coeff_flat = results_df_combine_flat(results_all_wavelets_thresh05_coeff, noDWT_all_features_list, drop = True)


In [27]:
results_all_wavelets_thresh005_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh005_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh01_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh01_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh015_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh015_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh02_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh02_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh025_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh025_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh03_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh03_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh035_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh035_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh04_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh04_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh045_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh045_coeff2, noDWT_all_features_list, drop = True)
results_all_wavelets_thresh05_coeff2_flat = results_df_combine_flat(results_all_wavelets_thresh05_coeff2, noDWT_all_features_list, drop = True)


In [ ]:
results_all_wavelets_thresh005_coeff

In [106]:
save_with_pickle("results_all_wavelets_thresh005_coeff_flat.pkl", results_all_wavelets_thresh005_coeff_flat)
save_with_pickle("results_all_wavelets_thresh01_coeff_flat.pkl", results_all_wavelets_thresh01_coeff_flat)
save_with_pickle("results_all_wavelets_thresh015_coeff_flat.pkl", results_all_wavelets_thresh015_coeff_flat)
save_with_pickle("results_all_wavelets_thresh02_coeff_flat.pkl", results_all_wavelets_thresh02_coeff_flat)
save_with_pickle("results_all_wavelets_thresh025_coeff_flat.pkl", results_all_wavelets_thresh025_coeff_flat)
save_with_pickle("results_all_wavelets_thresh03_coeff_flat.pkl", results_all_wavelets_thresh03_coeff_flat)
save_with_pickle("results_all_wavelets_thresh035_coeff_flat.pkl", results_all_wavelets_thresh035_coeff_flat)
save_with_pickle("results_all_wavelets_thresh04_coeff_flat.pkl", results_all_wavelets_thresh04_coeff_flat)
save_with_pickle("results_all_wavelets_thresh045_coeff_flat.pkl", results_all_wavelets_thresh045_coeff_flat)
save_with_pickle("results_all_wavelets_thresh05_coeff_flat.pkl", results_all_wavelets_thresh05_coeff_flat)


In [28]:
save_with_pickle("results_all_wavelets_thresh005_coeff2_flat.pkl", results_all_wavelets_thresh005_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh01_coeff2_flat.pkl", results_all_wavelets_thresh01_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh015_coeff2_flat.pkl", results_all_wavelets_thresh015_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh02_coeff2_flat.pkl", results_all_wavelets_thresh02_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh025_coeff2_flat.pkl", results_all_wavelets_thresh025_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh03_coeff2_flat.pkl", results_all_wavelets_thresh03_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh035_coeff2_flat.pkl", results_all_wavelets_thresh035_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh04_coeff2_flat.pkl", results_all_wavelets_thresh04_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh045_coeff2_flat.pkl", results_all_wavelets_thresh045_coeff2_flat)
save_with_pickle("results_all_wavelets_thresh05_coeff2_flat.pkl", results_all_wavelets_thresh05_coeff2_flat)


In [73]:
results_all_wavelets_thresh01_flat

,bior1.1,bior1.3,bior1.5,bior2.2,bior2.4,bior2.6,bior2.8,bior3.1,bior3.3,bior3.5,...,sym12,sym13,sym14,sym15,sym16,sym17,sym18,sym19,sym20,no_DWT
0,"[[70.84782893341436, -32.43689215251768, 2.184...","[[70.84782893341436, -33.82453236730736, 2.094...","[[70.84782893341436, -34.36405508916663, 2.061...","[[70.84782893341436, -33.352646270282854, 2.12...","[[70.84782893341436, -33.77536135052132, 2.097...","[[70.84782893341436, -33.7181452797496, 2.1011...","[[70.84782893341436, -33.80662095563988, 2.095...","[[70.84782893341436, -33.768181848267794, 2.09...","[[70.84782893341436, -33.4980795662407, 2.1149...","[[70.84782893341436, -34.706546145831034, 2.04...",...,"[[70.84782893341436, -33.78858056847753, 2.096...","[[70.84782893341436, -33.65002596606458, 2.105...","[[70.84782893341436, -33.81829332237512, 2.094...","[[70.84782893341436, -34.01668846802737, 2.082...","[[70.84782893341436, -33.7973000375961, 2.0962...","[[70.84782893341436, -33.99364261913832, 2.084...","[[70.84782893341436, -33.811250037907854, 2.09...","[[70.84782893341436, -34.00015529163682, 2.083...","[[70.84782893341436, -33.82461912325078, 2.094...","[[70.84782893341436, -36.37763497192012, 1.947..."


In [118]:
results_all_wavelets_thresh01_flat.to_pickle("results_all_wavelets_thresh01_flat.pkl")
results_all_wavelets_thresh02_flat.to_pickle("results_all_wavelets_thresh02_flat.pkl")
results_all_wavelets_thresh03_flat.to_pickle("results_all_wavelets_thresh03_flat.pkl")
results_all_wavelets_thresh04_flat.to_pickle("results_all_wavelets_thresh04_flat.pkl")


In [119]:
results_all_wavelets_thresh005_flat.to_pickle("results_all_wavelets_thresh005_flat.pkl")
results_all_wavelets_thresh015_flat.to_pickle("results_all_wavelets_thresh015_flat.pkl")
results_all_wavelets_thresh025_flat.to_pickle("results_all_wavelets_thresh025_flat.pkl")
results_all_wavelets_thresh035_flat.to_pickle("results_all_wavelets_thresh035_flat.pkl")
results_all_wavelets_thresh045_flat.to_pickle("results_all_wavelets_thresh045_flat.pkl")
results_all_wavelets_thresh05_flat.to_pickle("results_all_wavelets_thresh05_flat.pkl")


In [54]:
results_all_wavelets_thresh2005_flat.to_pickle("results_all_wavelets_thresh2005_flat.pkl")
results_all_wavelets_thresh201_flat.to_pickle("results_all_wavelets_thresh201_flat.pkl")
results_all_wavelets_thresh2015_flat.to_pickle("results_all_wavelets_thresh2015_flat.pkl")
results_all_wavelets_thresh202_flat.to_pickle("results_all_wavelets_thresh202_flat.pkl")
results_all_wavelets_thresh2025_flat.to_pickle("results_all_wavelets_thresh2025_flat.pkl")
results_all_wavelets_thresh203_flat.to_pickle("results_all_wavelets_thresh203_flat.pkl")
results_all_wavelets_thresh2035_flat.to_pickle("results_all_wavelets_thresh2035_flat.pkl")
results_all_wavelets_thresh204_flat.to_pickle("results_all_wavelets_thresh204_flat.pkl")
results_all_wavelets_thresh2045_flat.to_pickle("results_all_wavelets_thresh2045_flat.pkl")
results_all_wavelets_thresh205_flat.to_pickle("results_all_wavelets_thresh205_flat.pkl")


In [55]:
results_all_wavelets_thresh3005_flat.to_pickle("results_all_wavelets_thresh3005_flat.pkl")
results_all_wavelets_thresh301_flat.to_pickle("results_all_wavelets_thresh301_flat.pkl")
results_all_wavelets_thresh3015_flat.to_pickle("results_all_wavelets_thresh3015_flat.pkl")
results_all_wavelets_thresh302_flat.to_pickle("results_all_wavelets_thresh302_flat.pkl")
results_all_wavelets_thresh3025_flat.to_pickle("results_all_wavelets_thresh3025_flat.pkl")
results_all_wavelets_thresh303_flat.to_pickle("results_all_wavelets_thresh303_flat.pkl")
results_all_wavelets_thresh3035_flat.to_pickle("results_all_wavelets_thresh3035_flat.pkl")
results_all_wavelets_thresh304_flat.to_pickle("results_all_wavelets_thresh304_flat.pkl")
results_all_wavelets_thresh3045_flat.to_pickle("results_all_wavelets_thresh3045_flat.pkl")
results_all_wavelets_thresh305_flat.to_pickle("results_all_wavelets_thresh305_flat.pkl")


old

In [94]:
new_column_df = pd.DataFrame({'no_DWT': noDWT_all_features_list}, index=results_all_wavelets22.index)

df_all_wavelets_and_noDWT_features = results_all_wavelets22.merge(new_column_df,  left_index=True, right_index=True)
df_all_wavelets_and_noDWT_features

,bior1.1,bior1.3,bior1.5,bior2.2,bior2.4,bior2.6,bior2.8,bior3.1,bior3.3,bior3.5,...,sym12,sym13,sym14,sym15,sym16,sym17,sym18,sym19,sym20,no_DWT
DWT_5nm_2,"[[62.54401448636022, -32.87969339916074, 1.902...","[[65.49068465605141, -34.71013486059351, 1.886...","[[66.70070712369932, -35.61648614604331, 1.872...","[[72.73019359697264, -34.23824876356899, 2.124...","[[74.31464199164567, -35.027792407398, 2.12159...","[[74.52417689378515, -34.97057633662628, 2.131...","[[76.31468332513194, -35.57782594221214, 2.145...","[[63.16561281267428, -33.45522696869912, 1.888...","[[64.46037255232442, -33.46342466223302, 1.926...","[[65.27111125083384, -34.60250866936635, 1.886...",...,"[[71.68373356475912, -35.46107278732244, 2.021...","[[68.29867754296085, -35.34170464016569, 1.932...","[[71.85898532593403, -35.48910888520291, 2.024...","[[70.74134216584353, -36.51545053356663, 1.937...","[[71.0676330778248, -36.31184447098931, 1.9571...","[[70.81625712836355, -36.52745424592905, 1.938...","[[71.00833914819391, -36.34034190572744, 1.953...","[[70.73068506949345, -36.48414536888052, 1.938...","[[71.016688885416, -36.35030784884103, 1.95367...","[[70.84782893341436, -36.37763497192012, 1.947..."
DWT_5nm_3,"[[51.96531191877389, -23.58364508971054, 2.203...","[[54.709349027957714, -23.563387254022103, 2.3...","[[55.84574169089243, -24.493089106964106, 2.28...","[[52.99648459048414, -26.33301286792826, 2.012...","[[54.67935798124359, -26.182058777956385, 2.08...","[[54.79678140927346, -26.4285731541277, 2.0733...","[[55.46837932297721, -28.382729297908156, 1.95...","[[56.7936784969291, -24.005021483068912, 2.365...","[[56.68837366979322, -24.826696601962045, 2.28...","[[57.19603921799672, -25.78778976592726, 2.217...",...,"[[57.25534601640646, -27.9882962952955, 2.0456...","[[58.14477491086562, -25.05748326435315, 2.320...","[[56.73651479176554, -27.698922229267165, 2.04...","[[58.94396465220086, -26.655630658173017, 2.21...","[[58.469345102563274, -26.752299020365594, 2.1...","[[58.85134551462681, -26.66456004112393, 2.207...","[[58.514156409093744, -26.72606180299151, 2.18...","[[58.957074658135475, -26.772058042771544, 2.2...","[[58.54141705799145, -26.677567747954242, 2.19...","[[58.05949865235694, -26.89914698362939, 2.158..."
DWT_5nm_4,"[[62.8744034198934, -33.00116298791156, 1.9052...","[[65.8156765137967, -33.15052621123097, 1.9853...","[[67.02668303914874, -34.449076372039826, 1.94...","[[70.3182973449185, -35.47813623013423, 1.9820...","[[72.26972399807954, -36.04069974760871, 2.005...","[[72.74715373870782, -35.807860247046634, 2.03...","[[74.64909773212331, -35.85508914520353, 2.081...","[[63.32102444737831, -37.373713868939674, 1.69...","[[65.21653752520078, -37.023752699694576, 1.76...","[[65.85992360420495, -37.14603867252609, 1.772...",...,"[[71.26122589268408, -36.25298052678428, 1.965...","[[67.35969695234155, -37.54119682368685, 1.794...","[[70.39558786001531, -37.23743740134799, 1.890...","[[69.47655695091234, -36.258117469551834, 1.91...","[[70.18000607292403, -35.71791183040455, 1.964...","[[69.62728437794976, -36.23008151935335, 1.921...","[[70.08875606134019, -35.708494254047785, 1.96...","[[69.46723608423537, -36.200175368442125, 1.91...","[[70.12513048880436, -35.66916704466778, 1.965...","[[69.34733372973942, -35.973851591305454, 1.92..."
DWT_5nm_5,"[[53.98699993743971, -16.089423290738022, 3.35...","[[56.4510848639219, -17.641690455824385, 3.199...","[[57.54021307074894, -17.641345374516757, 3.26...","[[61.76070323581308, -18.472584007514538, 3.34...","[[63.97477158031276, -19.62531514519758, 3.259...","[[64.2622552437555, -19.487220619801093, 3.297...","[[65.8024429368694, -21.178636705456057, 3.107...","[[54.91014305562034, -17.273659768543325, 3.17...","[[55.95408961469176, -17.686855998338082, 3.16...","[[56.53144078473984, -18.622276550094902, 3.03...",...,"[[62.034477164147376, -19.91989001972825, 3.11...","[[58.4688892274669, -19.896741446731696, 2.938...","[[62.00944034920584, -19.789145252667314, 3.13...","[

In [95]:
df_all_wavelets_and_noDWT_features.to_pickle("df_all_wavelets_and_noDWT_features.pkl")

In [108]:
new_column_df_thresh = pd.DataFrame({'no_DWT': noDWT_all_features_list}, index=results_all_wavelets22.index)

df_all_wavelets_and_noDWT_features_thresh = results_all_wavelets_thresh01.merge(new_column_df_thresh,  left_index=True, right_index=True)
df_all_wavelets_and_noDWT_features_thresh

,bior1.1,bior1.3,bior1.5,bior2.2,bior2.4,bior2.6,bior2.8,bior3.1,bior3.3,bior3.5,...,sym12,sym13,sym14,sym15,sym16,sym17,sym18,sym19,sym20,no_DWT
DWT_5nm_2,"[[62.98681573300329, -32.43689215251768, 1.941...","[[64.6050821627653, -33.82453236730736, 1.9100...","[[65.44827606682264, -34.36405508916663, 1.904...","[[72.73019359697264, -33.352646270282854, 2.18...","[[73.06221093476898, -33.77536135052132, 2.163...","[[73.27174583690845, -33.7181452797496, 2.1730...","[[74.54347833855967, -33.80662095563988, 2.204...","[[62.93827333738428, -33.768181848267794, 1.86...","[[63.77584679701588, -33.4980795662407, 1.9038...","[[64.01868019395718, -34.706546145831034, 1.84...",...,"[[70.05791801714614, -33.78858056847753, 2.073...","[[65.85904579470697, -33.65002596606458, 1.957...","[[70.17046212750132, -33.81829332237512, 2.074...","[[68.17614312879594, -34.01668846802737, 2.004...","[[68.67935556877144, -33.7973000375961, 2.0320...","[[68.65954926946219, -33.99364261913832, 2.019...","[[68.96908704545088, -33.811250037907854, 2.03...","[[68.11443786290316, -34.00015529163682, 2.003...","[[68.62337473008121, -33.82461912325078, 2.028...","[[70.84782893341436, -36.37763497192012, 1.947..."
DWT_5nm_3,"[[52.32818604520823, -23.220770963276212, 2.25...","[[53.98360077508908, -24.289135506890755, 2.22...","[[54.819378668816995, -23.466726084888684, 2.3...","[[52.90576605887556, -25.97013874149394, 2.037...","[[53.65299495916816, -26.182058777956385, 2.04...","[[53.77041838719804, -26.4285731541277, 2.0345...","[[54.01688281723992, -26.93123279217086, 2.005...","[[56.62992648644902, -24.255974258119824, 2.33...","[[56.258150565856255, -25.031955754244606, 2.2...","[[56.86327308193322, -24.855388607104313, 2.28...",...,"[[55.665490336085114, -25.917987391142113, 2.1...","[[56.471115004086045, -23.76145733442058, 2.37...","[[54.95209445971223, -25.795825943401912, 2.13...","[[56.96194904596701, -24.563531180486585, 2.31...","[[56.39086988720068, -24.098710052058447, 2.33...","[[57.25936322948991, -24.211202460113835, 2.36...","[[56.476693044983165, -24.408807708666675, 2.3...","[[56.875008790443246, -24.723448797876202, 2.3...","[[56.48797133765684, -24.047942772542925, 2.34...","[[58.05949865235694, -26.89914698362939, 2.158..."
DWT_5nm_4,"[[63.30782827417196, -32.567738133633014, 1.94...","[[64.94882680523958, -34.017375919788094, 1.90...","[[65.80077242476808, -33.223165757659174, 1.98...","[[70.3182973449185, -34.61128652157711, 2.0316...","[[71.04381338369888, -34.81478913322807, 2.040...","[[71.52124312432716, -34.58194963266598, 2.068...","[[72.91539831500906, -34.12138972808928, 2.136...","[[63.55612034278749, -36.93120682710329, 1.720...","[[64.52020277159691, -37.11602478781247, 1.738...","[[65.59513747643534, -35.934494198157715, 1.82...",...,"[[69.45374123271516, -34.15642081959127, 2.033...","[[65.20626982667991, -35.7082667335024, 1.8260...","[[68.30433767212392, -35.102665144690434, 1.94...","[[66.84402579724755, -33.822428690487655, 1.97...","[[67.85067216180413, -33.5097118153711, 2.0248...","[[67.27359011056784, -33.37780033921998, 2.015...","[[68.09721436358097, -33.24015290756917, 2.048...","[[66.70441179107593, -34.081591172962874, 1.95...","[[67.80622363497997, -33.48518842583782, 2.024...","[[69.34733372973942, -35.973851591305454, 1.92..."
DWT_5nm_5,"[[53.607866503144045, -16.468556725033675, 3.2...","[[55.69281799533059, -16.88342358723308, 3.298...","[[56.46786178128895, -17.531975747229442, 3.22...","[[61.76070323581308, -17.714317138923228, 3.48...","[[62.902420290852774, -18.552963855737573, 3.3...","[[63.18990395429551, -18.41486933034109, 3.431...","[[64.28590919968678, -19.662102968273448, 3.26...","[[54.719942076178754, -17.53703510965581, 3.12...","[[55.3679878310101, -17.874941725508187, 3.097...","[[55.45908949527984, -19.11656347258037, 2.901...",...,"[[60.64888790039639, -18.38562910255949, 3.298...","[[56.36918650919859, -18.61357105908221, 3.028...","[[60.229847101791364, -18.28022344805616, 3.29...","[

In [109]:
df_all_wavelets_and_noDWT_features_thresh.to_pickle("df_all_wavelets_and_noDWT_features_thresh.pkl")